## Using Python to Query MySQL
This notebook demonstrates using a couple of different database connectivity libraries to connect to and query a MySQL database.
- **PyMySQL** library
- MySQL's Native **mysql.connector** library
- **SqlAlchemy** library

### 1.0. Prerequisites

#### 1.1. First, you must **install** the libaries into your *python* environment by executing the following commands in a *Terminal window*
- \$ python3 -m pip install PyMySQL
- \$ python3 -m pip install mysql.connector
- \$ python3 -m pip install sqlalchemy

#### 1.2. Next, as with all Jupyter Notebooks, you need to **Import** the libaries that you'll be working with in the notebook,

In [1]:
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

import pandas as pd
import matplotlib.pyplot as plt

#### 1.3. And then, Assign Connection Variables for the MySQL Server & Database with which You'll be Working 

In [2]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "jtupitza"
pwd = "Passw0rd123"
db_name = "northwind"

### 2.0. Using the PyMySQL Library
#### 2.1. Using a Cursor to Iterate the Rows Returned

In [3]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute('SELECT * FROM products;')
    
    for row in cursor.fetchmany(size=5):
        print(row)
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")
    
conn.close()

('4', 1, 'NWTB-1', 'Northwind Traders Chai', None, Decimal('13.5000'), Decimal('18.0000'), 10, 40, '10 boxes x 20 bags', 0, 10, 'Beverages', b'')
('10', 3, 'NWTCO-3', 'Northwind Traders Syrup', None, Decimal('7.5000'), Decimal('10.0000'), 25, 100, '12 - 550 ml bottles', 0, 25, 'Condiments', b'')
('10', 4, 'NWTCO-4', 'Northwind Traders Cajun Seasoning', None, Decimal('16.5000'), Decimal('22.0000'), 10, 40, '48 - 6 oz jars', 0, 10, 'Condiments', b'')
('10', 5, 'NWTO-5', 'Northwind Traders Olive Oil', None, Decimal('16.0125'), Decimal('21.3500'), 10, 40, '36 boxes', 0, 10, 'Oil', b'')
('2;6', 6, 'NWTJP-6', 'Northwind Traders Boysenberry Spread', None, Decimal('18.7500'), Decimal('25.0000'), 25, 100, '12 - 8 oz jars', 0, 25, 'Jams, Preserves', b'')


In [4]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor(pymysql.cursors.DictCursor)

try:
    cursor.execute('SELECT * FROM products ORDER BY list_price DESC LIMIT 5;')
    
    for row in cursor.fetchall():
        print(row)
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")
    
conn.close()

{'supplier_ids': '2;6', 'id': 20, 'product_code': 'NWTJP-6', 'product_name': 'Northwind Traders Marmalade', 'description': None, 'standard_cost': Decimal('60.7500'), 'list_price': Decimal('81.0000'), 'reorder_level': 10, 'target_level': 40, 'quantity_per_unit': '30 gift boxes', 'discontinued': 0, 'minimum_reorder_quantity': 10, 'category': 'Jams, Preserves', 'attachments': b''}
{'supplier_ids': '2', 'id': 51, 'product_code': 'NWTDFN-51', 'product_name': 'Northwind Traders Dried Apples', 'description': None, 'standard_cost': Decimal('39.7500'), 'list_price': Decimal('53.0000'), 'reorder_level': 10, 'target_level': 40, 'quantity_per_unit': '50 - 300 g pkgs.', 'discontinued': 0, 'minimum_reorder_quantity': 10, 'category': 'Dried Fruit & Nuts', 'attachments': b''}
{'supplier_ids': '3;4', 'id': 43, 'product_code': 'NWTB-43', 'product_name': 'Northwind Traders Coffee', 'description': None, 'standard_cost': Decimal('34.5000'), 'list_price': Decimal('46.0000'), 'reorder_level': 25, 'target_lev

#### 2.2. Using the Pandas read_sql() Method to Return a DataFrame

In [5]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)

df = pd.read_sql("SELECT * FROM products ORDER BY list_price DESC;", conn)

conn.close()
df.head()

supplier_ids  id product_code                      product_name description  \
0          2;6  20      NWTJP-6       Northwind Traders Marmalade        None   
1            2  51    NWTDFN-51    Northwind Traders Dried Apples        None   
2          3;4  43      NWTB-43          Northwind Traders Coffee        None   
3            8   8       NWTS-8     Northwind Traders Curry Sauce        None   
4            6  17    NWTCFV-17  Northwind Traders Fruit Cocktail        None   

   standard_cost  list_price  reorder_level  target_level quantity_per_unit  \
0          60.75        81.0             10            40     30 gift boxes   
1          39.75        53.0             10            40  50 - 300 g pkgs.   
2          34.50        46.0             25           100   16 - 500 g tins   
3          30.00        40.0             10            40   12 - 12 oz jars   
4          29.25        39.0             10            40          15.25 OZ   

   discontinued  minimum_reorder_quantity                   category  \
0             0                      10.0            Jams, Preserves   
1             0                      10.0         Dried Fruit & Nuts   
2             0                      25.0                  Beverages   
3             0                      10.0                     Sauces   
4             0                      10.0  Canned Fruit & Vegetables   

  attachments  
0         b''  
1         b''  
2         b''  
3         b''  
4         b''

### 3.0. Using the MySQL Native Connection Library
#### 3.1. Using a Cursor to Iterate the Rows Returned

In [6]:
lbound = 15.00
ubound = 20.00

sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN %s AND %s
    ORDER BY list_price DESC;
"""

In [7]:
conn = mysql.connector.connect(user=user_id, password=pwd,
                               host=host_ip, database=db_name,
                               auth_plugin='mysql_native_password')

In [8]:
cursor = conn.cursor()

try:
    cursor.execute(sql_query, (lbound, ubound))
    
    for (product_id, product_name, list_price) in cursor:
        print("{}, {} was sold for {}".format(product_id, product_name, list_price))
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")   

57, Northwind Traders Ravioli was sold for 19.5000
40, Northwind Traders Crab Meat was sold for 18.4000
1, Northwind Traders Chai was sold for 18.0000
66, Northwind Traders Tomato Sauce was sold for 17.0000
86, Northwind Traders Cake Mix was sold for 15.9900


In [9]:
cursor = conn.cursor()

try:
    cursor.execute(sql_query, (lbound, ubound))
    
    row = cursor.fetchone() 
    while row:
        print(row)
        row = cursor.fetchone()
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")

(57, 'Northwind Traders Ravioli', Decimal('19.5000'))
(40, 'Northwind Traders Crab Meat', Decimal('18.4000'))
(1, 'Northwind Traders Chai', Decimal('18.0000'))
(66, 'Northwind Traders Tomato Sauce', Decimal('17.0000'))
(86, 'Northwind Traders Cake Mix', Decimal('15.9900'))


In [10]:
conn.close()

#### 3.2. Using Pandas read_sql() Method to Return a DataFrame

In [11]:
lbound = 15.00
ubound = 20.00

sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN {0} AND {1}
    ORDER BY list_price DESC;
""".format(lbound, ubound)

print(sql_query)


    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN 15.0 AND 20.0
    ORDER BY list_price DESC;



In [12]:
configs = {
    'user': user_id,
    'password': pwd,
    'host': host_name,
    'database': db_name,
    'auth_plugin': 'mysql_native_password',
    'raise_on_warnings': True
}

conn = mysql.connector.connect(**configs)

df = pd.read_sql(sql_query, conn)

conn.close()
df.tail()

product_id                    product_name  list_price
0          57       Northwind Traders Ravioli       19.50
1          40     Northwind Traders Crab Meat       18.40
2           1          Northwind Traders Chai       18.00
3          66  Northwind Traders Tomato Sauce       17.00
4          86      Northwind Traders Cake Mix       15.99

### 4.0. Using the SQLAlchemy Connection Library

In [13]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"

sqlEngine = create_engine(conn_str, pool_recycle=3600)
conn = sqlEngine.connect()

df = pd.read_sql(sql_query, conn);

conn.close()
df.head()

product_id                    product_name  list_price
0          57       Northwind Traders Ravioli       19.50
1          40     Northwind Traders Crab Meat       18.40
2           1          Northwind Traders Chai       18.00
3          66  Northwind Traders Tomato Sauce       17.00
4          86      Northwind Traders Cake Mix       15.99

### 5.0 Define Helper Functions to Encapsulate and Abstract the Implementation Details

In [14]:
sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    ORDER BY list_price DESC;
"""

#### 5.1. Using Individual Connection Parameters

In [15]:
def get_pymysql_dataframe(host, user, password, database_name, sql_query_string):
    connection = pymysql.connect(host=host, user=user, password=password, database=database_name)
    dframe = pd.read_sql(sql_query_string, connection)
    connection.close()
    
    return dframe

In [16]:
df = get_pymysql_dataframe(host_name, user_id, pwd, db_name, sql_query)
df.head()

product_id                      product_name  list_price
0          20       Northwind Traders Marmalade        81.0
1          51    Northwind Traders Dried Apples        53.0
2          43          Northwind Traders Coffee        46.0
3           8     Northwind Traders Curry Sauce        40.0
4          17  Northwind Traders Fruit Cocktail        39.0

In [17]:
print("Shape: {}\n".format(df.shape))

Shape: (45, 3)



#### 5.1.1. Using SqlAlchemy

In [18]:
def get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe

In [19]:
df = get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, sql_query)
df.head()

product_id                      product_name  list_price
0          20       Northwind Traders Marmalade        81.0
1          51    Northwind Traders Dried Apples        53.0
2          43          Northwind Traders Coffee        46.0
3           8     Northwind Traders Curry Sauce        40.0
4          17  Northwind Traders Fruit Cocktail        39.0

In [20]:
print(f"Shape: {df.shape[0]} Observations x {df.shape[1]} Features")

Shape: 45 Observations x 3 Features


#### 5.2. Using a Configurations Dictionary

In [21]:
def get_mysql_dataframe(sql_query_string, args):
    connection = mysql.connector.connect(**args)
    dframe = pd.read_sql(sql_query_string, connection)
    connection.close()
    
    return dframe

In [22]:
dframe = get_mysql_dataframe(sql_query, configs)
dframe.tail()

product_id                   product_name  list_price
40          89      Northwind Traders Peaches         1.5
41          94         Northwind Traders Peas         1.5
42          88        Northwind Traders Pears         1.3
43          92  Northwind Traders Green Beans         1.2
44          93         Northwind Traders Corn         1.2

In [32]:
print(f"Shape: {dframe.shape[0]} Observations x {dframe.shape[1]} Features")

Shape: 45 Observations x 3 Features


### 6.0. Writing a Pandas DataFrame to a SQL Database

In [37]:
def insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, table_name):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    df.to_sql(table_name, con=connection, if_exists='replace') #, index_label='product_id');  'append'
    connection.close()

In [38]:
insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, dframe, 'dim_products')

In [39]:
df = get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, 'SELECT * FROM dim_products')
df.head()

index  product_id                      product_name  list_price
0      0          20       Northwind Traders Marmalade        81.0
1      1          51    Northwind Traders Dried Apples        53.0
2      2          43          Northwind Traders Coffee        46.0
3      3           8     Northwind Traders Curry Sauce        40.0
4      4          17  Northwind Traders Fruit Cocktail        39.0

### 7.0. Explore Pandas DataFrames' Capabilities
#### 7.1. Display the Data Type of Each Feature

In [24]:
sql_query = "SELECT * FROM northwind.products;"

df = get_mysql_dataframe(sql_query, configs)

In [25]:
df.dtypes

supplier_ids                 object
id                            int64
product_code                 object
product_name                 object
description                  object
standard_cost               float64
list_price                  float64
reorder_level                 int64
target_level                  int64
quantity_per_unit            object
discontinued                  int64
minimum_reorder_quantity    float64
category                     object
attachments                  object
dtype: object

#### 7.1. Inspect the Cardinality (number of unique values) of each Feature

In [26]:
df.nunique()

supplier_ids                12
id                          45
product_code                43
product_name                45
description                  0
standard_cost               29
list_price                  37
reorder_level                8
target_level                10
quantity_per_unit           32
discontinued                 1
minimum_reorder_quantity     6
category                    16
attachments                  1
dtype: int64

In [27]:
unique_values = []

for column in df.columns:
    unique_values.append(df[column].unique())
    
data = list(zip(df.columns, unique_values))    
    
pd.DataFrame(data, columns=['Feature', 'Unique Values'])

Feature  \
0               supplier_ids   
1                         id   
2               product_code   
3               product_name   
4                description   
5              standard_cost   
6                 list_price   
7              reorder_level   
8               target_level   
9          quantity_per_unit   
10              discontinued   
11  minimum_reorder_quantity   
12                  category   
13               attachments   

                                        Unique Values  
0           [4, 10, 2;6, 2, 8, 6, 1, 7, 3;4, 5, 3, 9]  
1   [1, 3, 4, 5, 6, 7, 8, 14, 17, 19, 20, 21, 34, ...  
2   [NWTB-1, NWTCO-3, NWTCO-4, NWTO-5, NWTJP-6, NW...  
3   [Northwind Traders Chai, Northwind Traders Syr...  
4                                              [None]  
5   [13.5, 7.5, 16.5, 16.0125, 18.75, 22.5, 30.0, ...  
6   [18.0, 10.0, 22.0, 21.35, 25.0, 30.0, 40.0, 23...  
7                    [10, 25, 5, 15, 30, 20, 50, 100]  
8        [40, 100, 20, 60, 120, 80, 75, 125, 200, 50]  
9   [10 boxes x 20 bags, 12 - 550 ml bottles, 48 -...  
10                                                [0]  
11           [10.0, 25.0, 5.0, 15.0, 30.0, 20.0, nan]  
12  [Beverages, Condiments, Oil, Jams, Preserves, ...  
13                                              [b'']

#### 7.2. Display any Missing (NULL) values

In [28]:
df.isnull().sum().sort_values(ascending=True)

supplier_ids                 0
id                           0
product_code                 0
product_name                 0
standard_cost                0
list_price                   0
reorder_level                0
target_level                 0
discontinued                 0
category                     0
attachments                  0
quantity_per_unit            5
minimum_reorder_quantity    15
description                 45
dtype: int64

#### 7.3. Separate Numerical and Categorical Features

In [29]:
numerical_cols = [col for col in df.columns if df.dtypes[col] != 'O']
categorical_cols = [col for col in df.columns if col not in numerical_cols]

print(numerical_cols)
print(categorical_cols)

['id', 'standard_cost', 'list_price', 'reorder_level', 'target_level', 'discontinued', 'minimum_reorder_quantity']
['supplier_ids', 'product_code', 'product_name', 'description', 'quantity_per_unit', 'category', 'attachments']


#### 7.4. Evaluate the Statistical Distribution of the Numerical Features

In [30]:
df[numerical_cols].describe()

id  standard_cost  list_price  reorder_level  target_level  \
count  45.000000      45.000000   45.000000      45.000000     45.000000   
mean   57.933333      11.682500   15.845778      22.444444     69.555556   
std    33.750017      12.689461   16.743022      23.442924     50.506775   
min     1.000000       0.500000    1.200000       5.000000     20.000000   
25%    21.000000       2.000000    2.990000      10.000000     40.000000   
50%    66.000000       7.500000   10.000000      10.000000     40.000000   
75%    88.000000      16.012500   21.350000      25.000000    100.000000   
max    99.000000      60.750000   81.000000     100.000000    200.000000   

       discontinued  minimum_reorder_quantity  
count          45.0                 30.000000  
mean            0.0                 15.000000  
std             0.0                  8.304548  
min             0.0                  5.000000  
25%             0.0                 10.000000  
50%             0.0                 10.000000  
75%             0.0                 25.000000  
max             0.0                 30.000000

#### 7.5. Write the Contents of the DataFrame to a Comma-Separated Values (CSV) File

In [31]:
data_dir = os.path.join(os.getcwd(), 'data')
dest_file = os.path.join(data_dir, 'northwind_products.csv')

df.to_csv(dest_file)